

### Progettazione e realizzazione di job in MapReduce, Hive e Spark


Progetto a cura del gruppo **Data-FaSt** (Farioli Davide, Stojani Marjo)

### Indice trattazione




**1. Specifiche Hardware e Software**
  
    
**2. Specifiche dataset**


**3. Realizzazione Job1**

    3.1 Specifiche
    
    3.2 MapReduce
    
    3.3 Hive
    
    3.4 Spark
    
    3.5 Risultati
    
    3.6 Grafici
    
    
**4. Realizzazione Job2**

    4.1 Specifiche
    
    4.2 MapReduce
    
    4.3 Hive
    
    4.4 Spark
    
    4.5 Risultati
    
    4.6 Grafici
    

**5. Realizzazione Job3**

    5.1 Specifiche
    
    5.2 MapReduce
    
    5.3 Hive
    
    5.4 Spark
    
    5.5 Risultati
    
    5.6 Grafici
    
**6. Conclusioni**

## 1. Specifiche Hardware e Software

L'esecuzione dei Job è stata svolta sia in macchina locale che su Cluster AWS

I Job in MapReduce e Spark sono stati realizzati con il linguaggio di programmazione **Python**

### Macchina Locale
#### Specifiche Hardware:
- **Processor:** Intel Core i5 4th Gen 4310U

- **Memory:** 8GB

- **Storage:** 256GB (SSD)

#### Specifiche Software:
- **SO:** Linux Mint 19
- **Python:** 3.7.*
- **Java:** 1.8
- **Hadoop:** 3.2.1
- **Hive:** 2.3.7
- **Spark:** 3.0.0

### Cluster AWS
#### Specifiche Hardware:
- **Istanza EMR:** m5.xlarge

- **Nodes:** 1 Master, 2 Core

- **Storage:** S3 bucket

#### Specifiche Software:
- **Python:** 3.7.*
- **Java:** 1.8
- **Hadoop:** 3.2.1
- **Hive:** 3.1.2
- **Spark:** 3.0.0

## 2. Specifiche dataset

Il Dataset di riferimento **Daily Historical Stock Prices** contiene l’andamento giornaliero di
un’ampia selezione di azioni sulla borsa di New York (NYSE) e sul NASDAQ dal 1970 al 2018. 
Il dataset è formato da due file CSV: historical_stock_prices.csv e historical_stocks.csv


**historical_stock_prices.csv** è composto da 20973889 righe e 8 colonne con i seguenti campi:

- **ticker:** simbolo univoco dell’azione

- **open:** prezzo di apertura

- **close:** prezzo di chiusura

- **adj_close:** prezzo di chiusura modificato

- **low:** prezzo minimo

- **high:** prezzo massimo

- **volume:** numero di transazioni

- **date:** data nel formato aaaa-mm-gg

il file non presenta campi con valore nullo

il minimo valore nel campo 'date' è: '1970-01-02'

il massimo valore nel campo 'date' è: '2018-08-24'

In [19]:
historical_stock_prices.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


**historical_stocks.csv** è composta da 6460 righe e 5 colonne con i seguenti campi:

- **ticker:** simbolo univoco dell’azione

- **exchange:** NYSE o NASDAQ

- **name:** nome dell'azienda

- **sector:** settore dell'azienda

- **industry:** industria di riferimento per l'azienda

il file presenta i campi 'sector' e 'industry' con valori nulli

ad un'azienda (campo 'name') possono essere associati anche diversi ticker

un'azienda può essere associata a diversi settori

In [22]:
historical_stocks.head()

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS


## 3. Realizzazione Job1
### 3.1 Specifiche
Realizzare un job che sia in grado di generare le statistiche di ciascuna azione tra il 2008 e il 2018 indicando, per ogni azione: il simbolo, la variazione della quotazione (differenza percentuale arrotondata tra i prezzi di chiusura iniziale e finale dell’intervallo temporale), il prezzo minimo, quello massimo e il volume medio nell’intervallo, ordinando l’elenco in ordine decrescente di variazione della quotazione

### 3.2 MapReduce
Linguaggio di programmazione utilizzato: **Python** 3.7

Per la realizzazione del Job1 in MapReduce è stato sufficiente 1 stage, sono quindi stati implementati 1 mapper e 1 reducer

#### Mapper
Per realizzare il Job1 è necessario utilizzare solo il file **historical_stock_prices.csv** dunque il mapper svolge solo un'operazione di filtraggio dei dati in base alla data del ticker, accettando solo quelle di interesse (comprese tra il 2008 e il 2018) e passando al reducer solo le colonne 'ticker,close,volume,date' di cui ticker sarà la chiave per lo shuffle and sort; in questa maniera vengono scartate tutte le righe e colonne di dati non necessarie ai fini dell'obiettivo del job riducendo sin da subito anche il traffico di dati all'interno del file system distribuito

##### Mapper: pseudocodifica



![job1_mapper](img/job1_mapper.png)

#### Reducer
Il Reducer prende in input i valori filtrati dal Mapper e riordinati dalla fase di shuffle and sort; ogni stream in input ha dunque i seguenti campi: 'ticker,close,volume,date'. Viene utilizzata una struttura dati di supporto per registrare i dati relativi ad ogni ticker in base al suo obiettivo: cumulare il volume per poi poterne ricavare la media complessiva, registrare il prezzo di chiusura con la data più piccola ed il prezzo di chiusura con la data più grande per poterne ricavare la variazione percentuale, registrare il prezzo minimo e il prezzo massimo.

##### Reducer: pseudocodifica

![job1_reducer](img/job1_reducer.png)

### 3.3 Hive

Per la realizzazione del Job1 in Hive è necessario creare una tabella dove vengono inseriti i dati del file in input **historical_stock_prices.csv**. Successivamente vengono create diverse Viste: 

nella Vista 'single_fields_statistics' vengono calcolati il volume medio, il prezzo minimo ed il prezzo massimo in modo semplice grazie al raggruppamento in base al ticker, ovviamente filtrando solo le righe per cui la data è compresa tra il 2008 e il 2018;

la Vista 'first_last_date' registra la minima e la massima data disponibile per ogni ticker, questa vista è utile per la creazione delle successive;

le Viste 'first_close' e 'last_close' registrano il prezzo relativo rispettivamente alla data minima e alla data massima per ogni ticker;

la Vista 'ticker_variation' calcola la variazione  percentuale per ogni ticker sfruttando i valori ricavati dal join delle viste 'first_close' e 'last_close';

la Vista 'job1_result' effettua un join tra la 'ticker_variation' e la 'single_fields_statistics' in base al ticker ottenendo così tutti i valori utili per l'output che si potranno ottenere con una semplice SELECT *

![job1_hive1](img/job1_hive.png)

### 3.4 Spark

![job2_spark](img/job1_spark.jpg)

### 3.5 Risultati

![job1_results](img/job1_results.png)

### 3.6 Grafico
Il grafico seguente mostra il tempo di esecuzione (l'ascissa indica il tempo in secondi) del Job1 differenziato per tecnologia utilizzata (mapReduce, Hive o Spark), grandezza del dataset in input (25%, 50% o 100%)e luogo fisico di esecuzione (macchina locale o Cluster AWS).

![job1_graphic](img/job1_graphic.jpg)

## 4. Realizzazione Job2
### 4.1 Specifiche
Realizzare un job che sia in grado di generare, per ciascun settore, il relativo “trend” nel periodo 2008-2018 ovvero un elenco contenete, per ciascun anno nell’intervallo: il volume annuale medio delle azioni del settore, la variazione annuale media delle aziende del settore e la quotazione giornaliera media delle aziende del settore.

### 4.2 MapReduce
Linguaggio di programmazione utilizzato: **Python** 3.7

Per la realizzazione del Job2 in MapReduce sono serviti 2 stage, sono quindi stati implementati 2 mapper e 2 reducer. Il primo stage necessario per il join dei due file di input, il secondo per l'elaborazione dell'output.

### - Stage 1
#### Mapper 1
Per realizzare il Job2 è necessario utilizzare sia il file **historical_stock_prices.csv** sia il file **historical_stocks.csv**; dunque il mapper dello stage 1, oltre a svolgere un'operazione di filtraggio dei dati in base alla data del ticker, svolge un'operazione di riconoscimento dell'origine dell'input, passaggio fondamentale per la cooperazione con il reducer che potrà trattare le righe di input in maniera adeguata per ricavare i dati da elaborare. Un modo per riconoscere a quale file di input appartiene lo stream in esame è controllare il numero dei campi, visto che i file di input hanno un numero di colonne diverso tra loro. L'output generato dal mapper sarà composto dal formato 'ticker,close,volume,date,sector' di cui i primi due campi compongono la chiave, strategia utilizzata per ordinare in base al ticker e passare come primo dato utile il settore del ticker.

##### Mapper 1: pseudocodifica


![job2_mapper1](img/job2_mapper1.png)

#### Reducer 1
Il Reducer dello stage 1 si occupa del effettivo join dei dati. Sfruttando la fase di shuffle and sort su due chiavi, il reducer riconosce il settore del ticker con il primo stream e lo associa a tutte le righe di dati del ticker

##### Reducer 1: pseudocodifica


![job2_reducer1](img/job2_reducer1.png)

### - Stage 2
#### Mapper 2
La fase di mapping dello stage 2 prevede una semplicissima lettura dei dati di input senza operare filtraggi mandando in output stream in formato 'sector,ticker,close,volume,date', in modo da operare il successivo shuffle and sort sul campo 'sector' 

##### Mapper 2: pseudocodifica


![job2_mapper2](img/job2_mapper2.png)

#### Reducer 2
Il Reducer dello stage 2 utilizza diverse strutture dati di supporto utili per le 2 principali fasi: calcolo dei valori medi annuali('volume', 'close' e variazione percentuale) per ogni ticker, calcolo della media annuale dei valori medi annuali di tutti i ticker di un settore. 

La prima dictionary ha come chiave la tupla (ticker,year) e come valori vengono cumulati il volume e il close per calcolarne la media, il close con data minima per quel anno e il close con data massima per quel anno.

La seconda dictionary ha come chiave la tupla (sector,year) e come valori vengono comulati il volume medio, il close medio e la variation media dei ticker associati al settore per quel anno.

##### Reducer 2: pseudocodifica


![job2_reducer2](img/job2_reducer2.png)

### 4.3 Hive

Per la realizzazione del Job2 in Hive è necessario creare due tabelle dove vengono inseriti i dati dei file in input **historical_stock_prices.csv** e **historical_stocks.csv**. Successivamente vengono create diverse Viste: 

nella Vista 'joined_dataset' viene effettuato il join tra le due tabelle appena popolate filtrando solo le righe per cui la data è compresa tra il 2008 e il 2018 e mantenendo solo le colonne di interesse: 'ticker,volume,close,ticker_year,sector'

la Vista 'avg_volume_close' utilizza la vista 'joined_dataset' per calcolare il volume e il close medio raggruppando per sector e year

le Vista 'min_date_year' e 'max_date_year' registrano la minima e la massima data disponibile per ogni year per ogni ticker, queste viste sono utili per la creazione delle successive viste 'min_close_year' e 'max_close_year' per ricavare i rispettivi close per poi ricavare la variazione annuale dei ticker nella view 'ticker_variation';

le Viste 'min_close_year' e 'max_close_year' registrano il prezzo relativo rispettivamente alla data minima e alla data massima per ogni ticker;

la Vista 'sector_variation' calcola la variazione percentuale media delle variazioni percentuali dei ticker per year, associandola al relativo sector;

la Vista 'job2_result' effettua un join tra la 'sector_variation' e la 'avg_volume_close' in base al sector ottenendo così tutti i valori utili per l'output che si potranno ottenere con una semplice SELECT *

![job2_hive](img/job2_hive.png)

### 4.4 Spark

![job2_spark](img/job2_spark.jpg)

### 4.5 Risultati

![job2_results](img/job2_results.png)

### 4.6 Grafici
Il grafico seguente mostra il tempo di esecuzione (l'ascissa indica il tempo in secondi) del Job2 differenziato per tecnologia utilizzata (mapReduce, Hive o Spark), grandezza del dataset in input (25%, 50% o 100%)e luogo fisico di esecuzione (macchina locale o Cluster AWS).

![job2_graphic](img/job2_graphic.jpg)

## 5. Realizzazione Job3
### 5.1 Specifiche
Realizzare un job in grado di generare gruppi di aziende le cui azioni hanno avuto lo stesso trend in termini di variazione annualenell’ultimo triennio disponibile, indicando le aziende e il trend comune (es. {Apple, Intel, Amazon}: 2016:-1%, 2017:+3%,2018:+5%).

### 5.2 MapReduce
Linguaggio di programmazione utilizzato: **Python** 3.7

Per la realizzazione del Job3 in MapReduce sono serviti 2 stage, sono quindi stati implementati 2 mapper e 2 reducer. Il primo stage necessario per il join dei due file di input, il secondo per l'elaborazione dell'output.

### - Stage 1
#### Mapper 1
Per realizzare il Job3 è necessario anche qui utilizzare sia il file **historical_stock_prices.csv** sia il file **historical_stocks.csv**; dunque il mapper dello stage 1, oltre a svolgere un'operazione di filtraggio dei dati in base alla data del ticker, svolge un'operazione di riconoscimento dell'origine dell'input. L'output generato dal mapper sarà composto dal formato 'ticker,close,date,name'.

##### Mapper 1: pseudocodifica

![job3_mapper1](img/job3_mapper1.png)

#### Reducer 1
Il Reducer dello stage 1 si occupa del effettivo join dei dati. Sfruttando una struttura dati con chiave (ticker,year) registra i 'close' con data minima e data massima per un certo ticker in un certo anno, in questo modo si riesce a calcolare la variazione percentuale annuale dei ticker sin dal primo reducer in modo da diminuire il flusso di dati toale. L'output generato sarà così composto 'name,ticker,year,variation' quindi come chiave avremo il nome dell'azienda

##### Reducer 1: pseudocodifica


![job3_reducer1](img/job3_reducer1.png)

### - Stage 2
#### Mapper 2
La fase di mapping dello stage 2 prevede una semplicissima lettura dei dati di input senza operare filtraggi mandando in output stream in formato 'name,ticker,close,volume,date', in modo da operare il successivo shuffle and sort sul campo 'name' riferito al nome dell'azienda

##### Mapper 2: pseudocodifica


![job3_mapper2](img/job3_mapper2.png)

#### Reducer 2
Il Reducer dello stage 3 utilizza diverse strutture dati di supporto utili per le 3 principali fasi: calcolo della variazione percentuale media annua per ogni azienda, confronto con le variazioni percentuali annue delle altre aziende e raggruppamento di aziende con trend uguali

La prima dictionary ha come chiave la tupla (company,year) e come valori vengono cumulate le variazioni percentuali dei ticker associati all'azienda per calcolarne la media.

La seconda dictionary ha come chiave 'company' e come valore una lista delle variazioni percentuali medie calcolate e posizionate in ordine annuo in modo tale da poter confrontare direttamente le liste nella fase successiva per poter trovare le aziende con trend uguali.

La terza struttura dati è una lista di liste, dove il campo 1 viene popolato con una lista di aziende con trend uguali e il campo 2 contiene il relativo trend in ordine annuo

##### Reducer 2: pseudocodifica


![job3_reducer2](img/job3_reducer2.png)

### 5.3 Hive

Per la realizzazione del Job3 in Hive è necessario creare due tabelle dove vengono inseriti i dati dei file in input **historical_stock_prices.csv** e **historical_stocks.csv**. Successivamente vengono create diverse Viste: 

nella Vista 'joined_dataset' viene effettuato il join tra le due tabelle appena popolate filtrando solo le righe per cui la data è compresa tra il 2016 e il 2018 e mantenendo solo le colonne di interesse: 'ticker,volume,close,ticker_year,name'

le Vista 'min_date_year' e 'max_date_year' registrano la minima e la massima data disponibile per ogni year per ogni ticker, queste viste sono utili per la creazione delle successive viste 'min_close_year' e 'max_close_year' per ricavare i rispettivi close per poi ricavare la variazione annuale dei ticker nella view 'ticker_variation';

le Viste 'min_close_year' e 'max_close_year' registrano il prezzo relativo rispettivamente alla data minima e alla data massima per ogni ticker;

la Vista 'ticker_variation' calcola la variazione percentuale di ogni ticker per ogni anno;

la Vista 'company_variation' calcola la variazione percentuale media delle variazioni percentuali dei ticker per year, associandola alla relativa azienda;

la Vista 'company_trend' collezione le variazioni di ogni azienda in una lista in modo da avere i campi 'company' e 'trend', formato molto utile per la vista successiva;

la Vista 'company_trend' colleziona le variazioni di ogni azienda in una lista in modo da avere i campi 'company' e 'trend', formato molto utile per la vista successiva;

la Vista 'job3_result' sfrutta il formato della vista 'company_trend' per la ricerca delle aziende con trend uguali, basta infatti raggruppare per il campo 'trend' e collezionare le aziende associate in una lista.

![job3_hive1](img/job3_hive1.png)

### 5.4 Spark

![job3_spark](img/job3_spark.jpg)

### 5.5 Risultati

![job3_results](img/job3_results.png)

### 5.6 Grafici
Il grafico seguente mostra il tempo di esecuzione (l'ascissa indica il tempo in secondi) del Job3 differenziato per tecnologia utilizzata (mapReduce, Hive o Spark), grandezza del dataset in input (25%, 50% o 100%)e luogo fisico di esecuzione (macchina locale o Cluster AWS).

![job3_graphic](img/job3_graphic.jpg)

## 6. Conclusioni

La tabella seguente mostra i tempi di esecuzione (in secondi) di tutti i Job

![timings](img/timings.jpg)

Osservando tutti i grafici e la tabella finale si evince che:

- Tutti i job eseguiti in Hive hanno un tempo di esecuzione molto più alto rispetto ai medesimi eseguiti in MapReduce o Spark.

- In MapReduce, nella fase di Mapping la scelta delle chiavi su cui verranno eseguite le operazioni di shuffle and sort è fondamentale in termini di tempo

- I vantaggi delle esecuzioni dei job su Cluster aumentano col crescere del dataset in input, per tutte le tecnologie utilizzate.

